In [ ]:
%pip install azure-cognitiveservices-speech



In [ ]:
import os
import pandas as pd
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials2.env")

def printmd(string):
    display(Markdown(string))

In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk
import openai

In [ ]:
file_url = "./data/MadeUpBookings.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [ ]:
openai.api_type = "azure"
openai.api_base = "https://openaisvcdataaidemos.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
SPEECH_KEY = os.getenv("AZURE_SPEECH_KEY")
SPEECH_REGION = os.getenv("AZURE_SPEECH_REGION")

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION)


In [ ]:
audio_output_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)

In [ ]:
# Should be the locale for the speaker's language.
speech_config.speech_recognition_language="en-US"
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

# The language of the voice that responds on behalf of Azure OpenAI.
speech_config.speech_synthesis_voice_name='en-US-JennyMultilingualNeural'
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_output_config)


In [ ]:
speech_recognition_result = speech_recognizer.recognize_once_async().get()
print(speech_recognition_result.text)

In [ ]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns, get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
- If the methods tried do not give the same result, reflect and try again until you have two methods that have the same result. 
- If you still cannot arrive to a consistent result, say that you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. 
- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on a section that starts with: "\n\nExplanation:\n". In the explanation, mention the column names that you used to get to the final answer. 
"""

In [ ]:
QUESTION = speech_recognition_result.text

In [ ]:
# MODEL = "mtc-text-davinci-003" # options: GPT4 or chat-gpt or mtc-text-davinci-003

# response = openai.Completion.create(engine=MODEL, prompt=PROMPT, max_tokens=100)
# text = response['choices'][0]['text'].replace('\n', ' ').replace(' .', '.').strip()
# print('Azure OpenAI response:' + text)
# speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()
# print("Speech synthesized to speaker for text [{}]".format(text))


In [ ]:
llm = AzureChatOpenAI(deployment_name="GPT4", temperature=0, max_tokens=500)

In [ ]:
agent_executor = create_pandas_dataframe_agent(llm=llm,df=df,verbose=True)

In [ ]:
agent_executor.agent.allowed_tools

In [ ]:
printmd(agent_executor.agent.llm_chain.prompt.template)

In [ ]:
# We are doing a for loop to retry N times. This is because: 
# 1) GPT-4 is still in preview and the API is being very throttled and 
# 2) Because the LLM not always gives the answer on the exact format the agent needs and hence cannot be parsed

for i in range(5):
    try:
        response = agent_executor.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX) 
        break
    except:
        response = "Error too many failed retries"
        continue
        
print(response)

In [ ]:
def remove_words_after_word(sentence, word):
    words = sentence.split()
    new_sentence = []
    for w in words:
        if w == word:
            break
        new_sentence.append(w)
    return ' '.join(new_sentence)

text2 = response.replace('\n', ' ').replace(' .', '.').strip()
new_sentence = remove_words_after_word(text2, "Explanation")
print('Azure OpenAI response:' + new_sentence)

In [ ]:
speech_synthesis_result = speech_synthesizer.speak_text_async(new_sentence).get()
print("Speech synthesized to speaker for text [{}]".format(new_sentence))